# Import

In [1]:
import json
import os
import sys
from datetime import datetime, timedelta

import pandas as pd

from src.kobis import (get_url, fetch_data)
from src.kobis import DB
import src.mongos as mg

%load_ext autoreload
%autoreload 2

# Const

In [2]:
collection = 'daily_boxoffice'
date_origin = '20031111'  # check http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=430156241533f1d058c603178cc3ca0e&targetDt=20031111
fmt = '%Y%m%d'

# Parameters for papermill

In [3]:
start_date = (datetime.today() - timedelta(days=1)).strftime(fmt)
end_date = (datetime.today() - timedelta(days=1)).strftime(fmt)

# start_date = 20190430
# start_date = 20180813
# end_date = 20180813

# Functions

In [4]:
def get_boxoffice_list(dct):
    return dct.get('boxOfficeResult', {}).get('dailyBoxOfficeList', [])
#     return dct['boxOfficeResult']['dailyBoxOfficeList']


def add_days(sdt, days=1, fmt='%Y%m%d'):
    fdt = datetime.strptime(sdt, fmt)
    fdt += timedelta(days=days)
    return fdt.strftime(fmt)

# Execute

In [5]:
url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json'
params = {}
datelist = pd.date_range(
    start=str(start_date),
    end=str(end_date),
)

for d in datelist:
    targetDt = d.strftime(fmt)
    params['targetDt'] = targetDt
    url0 = get_url(url, params)
    data = fetch_data(url0)
    lst = get_boxoffice_list(data)
    for e in lst:
        e['targetDt'] = targetDt

    if lst:
        mg.to_mongo(lst, DB, collection, query={'targetDt': targetDt}, delete=True, dtype='list')
    else:
        print('Data Empty:', targetDt)

Deleted rows: 0
Inserted rows: 10
